In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from codecarbon import EmissionsTracker

# Désactivation des messages TensorFlow inutiles (infos/warnings)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Initialisation du suivi énergétique
tracker = EmissionsTracker(project_name="MLP_Augmented")
tracker.start()

print("Environnement configuré.")

In [ ]:
# Chargement des données d'entraînement
train_ds = tf.keras.utils.image_dataset_from_directory(
    'banana_sushi/train',
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=True 
)

# Chargement des données de validation
val_ds = tf.keras.utils.image_dataset_from_directory(
    'banana_sushi/val',
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=False 
)

class_names = train_ds.class_names
print(f"Classes trouvées : {class_names}")

In [ ]:
# Définition de la couche de Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Construction du Modèle MLP
model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    data_augmentation,
    
    # Mise à l'échelle des pixels (0-255 vers 0-1)
    layers.Rescaling(1./255),
    
    # Transformation de la matrice 2D en vecteur 1D
    layers.Flatten(),
    
    # Couches denses (le "Cerveau" du MLP)
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3), # Pour éviter que le modèle ne dépende trop de certains neurones
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    
    # Couche de sortie (4 classes)
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

model.summary()

In [ ]:
print("Lancement de l'entraînement MLP avec Data Augmentation...")
history = model.fit(train_ds, validation_data=val_ds, epochs=15)

In [ ]:
print("\n--- RÉSULTATS MLP AUGMENTÉ ---")
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = model.predict(images, verbose=0)
    y_true.extend(tf.argmax(labels, axis=1).numpy())
    y_pred.extend(tf.argmax(preds, axis=1).numpy())

# Affichage du rapport de classification
print(classification_report(y_true, y_pred, target_names=class_names))

# Arrêt et affichage du bilan carbone
emissions = tracker.stop()
print(f"\n[BILAN ÉNERGÉTIQUE] Consommation : {emissions:.6f} kg CO2")